In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Initialize seed trip matrix
seed_matrix = np.array([
    [  5,  40, 120,  30,  50,  60],
    [ 40,   5,  52,  55,  60, 100],
    [120,  52,  10,  25,  90,  30],
    [ 30,  55,  25,  10,  15,  45],
    [ 50,  60,  90,  15,  15,  55],
    [ 60, 100,  30,  45,  55,  20]
])

# Test for symmetry
np.array_equal(seed_matrix, seed_matrix.T)

True

In [3]:
# Initialize target productions/attractions
target_productions = np.array([800, 400, 400, 200,  500, 700])
target_attractions = np.array([200, 600, 400, 300, 1000, 500])

# Initial predicted productions and attractions
predicted_productions = seed_matrix.sum(axis=1)
predicted_attractions = seed_matrix.sum(axis=0)

In [4]:
# Create DataFrames for ease of readability
trip_matrix = pd.DataFrame(seed_matrix)

# Set row and column indices from 1 to 6
trip_matrix.index = range(1, 7)
trip_matrix.columns = range(1, 7)

print("Initial Seed Matrix:")
trip_matrix

Initial Seed Matrix:


,1,2,3,4,5,6
1,5,40,120,30,50,60
2,40,5,52,55,60,100
3,120,52,10,25,90,30
4,30,55,25,10,15,45
5,50,60,90,15,15,55
6,60,100,30,45,55,20


In [5]:
# Define convergence criteria and max iterations
convergence_criteria = (0.90, 1.10)
max_iterations = 999

for iteration in range(1, max_iterations + 1):
    print(f"\nITERATION {iteration}")

    # Calculate row and column factors
    row_factors = target_productions / predicted_productions
    col_factors = target_attractions / predicted_attractions

    # Update the trip matrix with average factor adjustments
    for r in range(trip_matrix.shape[0]):
        for c in range(trip_matrix.shape[1]):
            avg_factor = (row_factors[r] + col_factors[c]) / 2
            trip_matrix.iloc[r, c] *= avg_factor

    # Prepare DataFrames for display with specified formatting

    trip_matrix_display = trip_matrix.round(1).copy().applymap(lambda x: f"{x:.1f}".rstrip('0').rstrip('.'))

    predicted_productions_df = pd.DataFrame({'Predicted Pi': predicted_productions}, index=trip_matrix.index).applymap(lambda x: f"{x:.1f}")

    predicted_attractions_df = pd.DataFrame([predicted_attractions], columns=trip_matrix.columns, index=['Predicted Aj']).applymap(lambda x: f"{x:.1f}")

    target_productions_df = pd.DataFrame(target_productions, index=trip_matrix.index, columns=['Target Pi']).applymap(lambda x: f"{x:.1f}")

    target_attractions_df = pd.DataFrame([target_attractions], columns=trip_matrix.columns, index=['Target Aj']).applymap(lambda x: f"{x:.1f}")

    row_factors_df = pd.DataFrame(row_factors, index=trip_matrix.index, columns=['Fi'])
    col_factors_df = pd.DataFrame(col_factors, index=trip_matrix.columns, columns=['Fj']).T


    # Combine all dataframes
    combined_display = pd.concat([trip_matrix_display, predicted_productions_df,target_productions_df, row_factors_df, ], axis=1)

    # Append the predicted and target attractions rows and column factors
    combined_display = pd.concat([combined_display, predicted_attractions_df, target_attractions_df, col_factors_df], axis=0)

    # Display the updated matrix with row/column factors and targets
    print("Updated Trip Matrix with Targets and Row/Column Factors:")
    display(combined_display.style.set_properties(**{'border': '1px solid black'}))

    # Update predicted productions and attractions
    predicted_productions = trip_matrix.sum(axis=1).values
    predicted_attractions = trip_matrix.sum(axis=0).values

    # Check convergence
    row_factors_close = (convergence_criteria[0] <= row_factors) & (row_factors <= convergence_criteria[1])
    col_factors_close = (convergence_criteria[0] <= col_factors) & (col_factors <= convergence_criteria[1])

    if row_factors_close.all() and col_factors_close.all():
        print("\nConvergence achieved.")
        break
else:
    print("\n\nConvergence not achieved after maximum iterations.")

# Display final trip matrix rounded to the nearest whole number
final_trip_matrix_display = trip_matrix.round(0).applymap(lambda x: f"{x:.0f}")
print("\n\nFinal Trip Matrix (Rounded):")
display(final_trip_matrix_display.style.set_properties(**{'border': '1px solid black'}))


ITERATION 1
Updated Trip Matrix with Targets and Row/Column Factors:


,1,2,3,4,5,6,Predicted Pi,Target Pi,Fi
1,8.2,90.9,230.8,64.3,153.3,127.1,305.0,800.0,2.622951
2,38.8,8,65.1,81.1,143.7,144.7,312.0,400.0,1.282051
3,112.7,81.8,12.2,36.1,212.9,42.5,327.0,400.0,1.223242
4,26.5,83.4,29.2,13.9,34.6,61.3,180.0,200.0,1.111111
5,60.3,110.3,134,25.7,39.5,92.6,285.0,500.0,1.754386
6,87.4,209.1,52.2,88.3,158.6,38.7,310.0,700.0,2.258065
Predicted Aj,305.0,312.0,327.0,180.0,285.0,310.0,nan,nan,nan
Target Aj,200.0,600.0,400.0,300.0,1000.0,500.0,nan,nan,nan
Fj,0.655738,1.923077,1.223242,1.666667,3.508772,1.612903,nan,nan,nan



ITERATION 2
Updated Trip Matrix with Targets and Row/Column Factors:


,1,2,3,4,5,6,Predicted Pi,Target Pi,Fi
1,7.3,100.7,225,69.3,194.1,138,674.6,800.0,1.185885
2,27.7,7.4,51.9,73,156.5,131.5,481.5,400.0,0.830792
3,79,74.9,9.6,32,228.8,38.1,498.4,400.0,0.802597
4,18.6,76.4,22.9,12.3,37.2,54.8,249.0,200.0,0.803372
5,50.6,116.4,123.6,26.3,47.9,95.7,462.3,500.0,1.081544
6,74.4,222.8,48.8,91.5,194.3,40.4,634.3,700.0,1.103588
Predicted Aj,333.9,583.6,523.5,309.4,742.7,507.0,nan,nan,nan
Target Aj,200.0,600.0,400.0,300.0,1000.0,500.0,nan,nan,nan
Fj,0.599051,1.028174,0.764039,0.969584,1.346495,0.986259,nan,nan,nan



ITERATION 3
Updated Trip Matrix with Targets and Row/Column Factors:


...

,1,2,3,4,5,6,Predicted Pi,Target Pi,Fi
1,6.6,107.5,208.6,73.5,233.7,148.2,763.0,800.0,1.048435
2,21,6.8,40.6,66,162.6,120.7,429.0,400.0,0.932325
3,58.3,66.5,7.3,28.2,231.9,34,440.2,400.0,0.908615
4,14.3,70.7,18.1,11.3,39.2,51,210.2,200.0,0.951256
5,46.1,125.2,115.6,28.1,58.1,103.6,468.2,500.0,1.067994
6,64.4,228.7,43.3,93.2,225.4,41.7,689.3,700.0,1.015557
Predicted Aj,225.1,604.1,452.7,301.7,918.4,498.0,nan,nan,nan
Target Aj,200.0,600.0,400.0,300.0,1000.0,500.0,nan,nan,nan
Fj,0.888443,0.993235,0.883673,0.994446,1.088796,1.003969,nan,nan,nan



ITERATION 5
Updated Trip Matrix with Targets and Row/Column Factors:


,1,2,3,4,5,6,Predicted Pi,Target Pi,Fi
1,6.5,108.5,203.5,74.5,243,150.4,778.0,800.0,1.028249
2,20.1,6.6,38.2,64.6,163.4,118.2,417.8,400.0,0.957388
3,55,64.2,6.8,27.3,230.8,33,426.1,400.0,0.938776
4,13.8,69.5,17.2,11.2,39.8,50.5,204.6,200.0,0.977340
5,46.1,127.7,113.9,28.8,61.1,106.2,476.7,500.0,1.048940
6,62.9,228.2,41.8,93.4,231.8,41.9,696.8,700.0,1.004622
Predicted Aj,210.7,605.3,433.6,300.3,951.0,499.1,nan,nan,nan
Target Aj,200.0,600.0,400.0,300.0,1000.0,500.0,nan,nan,nan
Fj,0.949103,0.991262,0.922570,0.999094,1.051495,1.001773,nan,nan,nan



Convergence achieved.


Final Trip Matrix (Rounded):


,1,2,3,4,5,6
1,7,109,203,74,243,150
2,20,7,38,65,163,118
3,55,64,7,27,231,33
4,14,70,17,11,40,50
5,46,128,114,29,61,106
6,63,228,42,93,232,42
